# Creating different Unet models in tensorflow

> different type of `unet model` from tversky focal loss paper will be implemented here

In [1]:
#| default_exp tf_model_creation

In [21]:
#| export
import tensorflow as tf
import numpy as np
import pandas as pd
from fastcore.all import *
from fastcore.test import *
from pathlib import Path
from typing import List, Tuple, Dict, Union, Callable, Optional
import os

In [2]:

print(os.environ.get('CUDA_PATH'))

/usr/lib/cuda


In [3]:
#| export
def pooling(
        inputs,
        max_pool_only=False,
        both=True,
        pool_size=2):
    if both:
        p1 = tf.keras.layers.MaxPooling2D(
            (pool_size, pool_size))(inputs)
        p2 = tf.keras.layers.AvgPool2D(
            pool_size=(pool_size, pool_size))(inputs)
        return tf.keras.layers.concatenate([p1, p2])
    elif max_pool_only:
        return tf.keras.layers.MaxPooling2D(
            (pool_size, pool_size))(inputs)
    else:
        return tf.keras.layers.AvgPool2D(
            pool_size=(pool_size, pool_size))(inputs)
            


In [4]:
# | export
def conv_block(
        inputs, 
        filter_no, 
        kernel_size, 
        batch_nm=True, 
        dropout=True, 
        drp_rt=0.1,
        kernel_initializer='glorot_normal' #option he_normal
        ):
    'Create a conv block with batch norma and dropout'
    c1 = tf.keras.layers.Conv2D(
        filter_no,
        (kernel_size, kernel_size),

        kernel_initializer='glorot_normal',
        padding='same',
        activation=None)(inputs)
    if batch_nm:
        c1 = tf.keras.layers.BatchNormalization()(c1)
    c1 = tf.keras.layers.Activation('relu')(c1)
    if dropout:
        c1 = tf.keras.layers.Dropout(drp_rt)(c1)
    return c1

In [5]:
input_size = tf.random.normal((1, 1052, 1632, 1))
out_ = conv_block(input_size, 32, 3, batch_nm=False, dropout=False)
test_eq([1,1052, 1632, 32], out_.shape)

2023-11-25 11:35:12.108532: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-25 11:35:12.140230: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-25 11:35:12.140500: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [9]:
#| export
def double_conv(
        inputs, 
        filter_no, 
        kernel_size, 
        batch_nm=True, 
        dropout=True, 
        drp_rt=0.1,
        kernel_initializer='glorot_normal' #option he_normal
        ):
    'Create double conv block with batch norma and dropout'

    c1 = tf.keras.layers.Conv2D(
        filter_no,
        (kernel_size, kernel_size),

        kernel_initializer='glorot_normal',
        padding='same',
        activation=None)(inputs)
    if batch_nm:
        c1 = tf.keras.layers.BatchNormalization()(c1)
    c1 = tf.keras.layers.Activation('relu')(c1)
    if dropout:
        c1 = tf.keras.layers.Dropout(drp_rt)(c1)
    c2 = tf.keras.layers.Conv2D(
        filter_no,
        (kernel_size, kernel_size),

        kernel_initializer='glorot_normal',
        padding='same',
        activation=None)(c1)
    if batch_nm:
        c2 = tf.keras.layers.BatchNormalization()(c2)
    c2 = tf.keras.layers.Activation('relu')(c2)
    if dropout:
        c2 = tf.keras.layers.Dropout(drp_rt)(c2)
    return c2

In [18]:
out_c = double_conv(inputs=input_size, filter_no=32, kernel_size=3, batch_nm=True, dropout=True, drp_rt=0.1)
out_p = pooling(out_c, both=True)


TensorShape([1, 526, 816, 64])

In [20]:
test_eq([1,1052, 1632, 32], out_c.shape)
test_eq([1,526, 816, 64], out_p.shape)

In [45]:
#| export
def encoder_block(
        input_size:tf.Tensor,
        filter_size:List[int]=[32,64,128,256,512, 1024],
        kernel_initializer:str='glorot_normal' #option he_normal
                 ):
    'Create Encoder block for Unet'

    inputs = tf.keras.layers.Input(input_size)
    outputs = []
    p = inputs 
    

    for i in filter_size:
        c = double_conv(
            inputs=p, 
            filter_no=i, 
            kernel_size=3, 
            batch_nm=True, 
            dropout=True, 
            drp_rt=0.1,
            kernel_initializer='glorot_normal' #option he_normal
            )
        p = pooling(c, both=True)
        outputs.append(p)
    return inputs, outputs

In [35]:
h, w, c = 1052, 1632, 1
input_img = tf.random.normal((1, h, w, c))
inputs,enc_outputs = encoder_block(
    input_size=(h, w, c),
    filter_size=[32,64,128,256,512, 1024],
    kernel_initializer='glorot_normal' #option he_normal
    )

In [41]:
test_eq((None, 1052, 1632, 1), inputs.shape)

In [44]:
expected_shapes = [
    (None, 526, 816, 64),
    (None, 263, 408, 128),
    (None, 131, 204, 256),
    (None, 65, 102, 512),
    (None, 32, 51, 1024),
    (None, 16, 25, 2048),
]

for (expected, actual) in zip(expected_shapes, enc_outputs):
    test_eq(expected, actual.shape)

In [49]:
enc_outputs[-1]

<KerasTensor: shape=(None, 16, 25, 2048) dtype=float32 (created by layer 'concatenate_19')>

In [47]:
len(enc_outputs)

6

In [46]:
filter_list=[32,64,128,256,512, 1024]
decoder_fl = filter_list[::-1]
decoder_fl


[1024, 512, 256, 128, 64, 32]

In [51]:
#| export
def decoder_block(
    filter_list:List[int],
    encoder_outputs:List[tf.Tensor],
   ):
    'Create a decoder block for unet'

    filter_list = filter_list[::-1]
    decoder_input = encoder_outputs[-1]

    for i in range(1, len(encoder_outputs)):
        filter_no = filter_list[i-1]
        up_ = tf.keras.layers.Conv2DTranspose(
                                            filter_no,
                                            (2,2),
                                            strides=(2,2),
                                            padding='same',
                                            )(decoder_input)
        
        # Handle the shape mismatch (if any)
        if up_.shape[1:3] != encoder_outputs[-(i + 1)].shape[1:3]:
            # Adjust padding, cropping, or use other techniques as needed
            print(f'up_.shape: {up_.shape}')
            pass

        # Concatenate the upsampled feature map with the corresponding encoder feature map
        up_ = tf.keras.layers.concatenate([up_, encoder_outputs[-(i + 1)]])

        c_= double_conv(
            inputs=up_, 
            filter_no=filter_no, 
            kernel_size=3, 
            batch_nm=True, 
            dropout=True, 
            drp_rt=0.1,
            kernel_initializer='glorot_normal' #option he_normal
            )
        decoder_input = c_
    return decoder_input


    



In [52]:
#| hide
import nbdev; nbdev.nbdev_export()